# IMAGE PROCESSING #

1. Import Texas School geo data
2. Create 30cm TIF file for each school in Austin ISD
3. ZERO SHOT BUILDING DETECTION (completed in another Studio)
4. Get TIF for each bounding box of the buildings detected

NOTES: use 30cm resolution for high-level address capture; higher resolution uses too much memory and is too slow to process ... higher resolution only necessary after buildings are identified, so much smaller per image memory requirement

In [1]:
import os
import pandas as pd
from PIL import Image
import geopandas as gpd
from tqdm.auto import tqdm
import leafmap

from schools import generate_tif_files, tif_to_png

In [13]:
%env ROOT_FOLDER=/home/Ryan.Skene/rooftop

env: ROOT_FOLDER=/home/Ryan.Skene/rooftop


In [25]:
LOW_RES_IMAGE_RESOLUTION = .3
HIGH_RES_IMAGE_RESOLUTION = .03
root_folder = os.environ['ROOT_FOLDER']
data_folder = os.path.join(root_folder, 'data')
images_folder = os.path.join(data_folder, 'images')
schools_folder = os.path.join(images_folder, 'schools')
school_geojson_file = os.path.join(data_folder, 'school_geocodes.geojson')
school_csv_file = os.path.join(data_folder, 'tx_schools.csv')

Filter Texas School Data - currenlty focused on Austin ISD

In [3]:
schools_geo = gpd.read_file('data/school_geocodes.geojson')
schools = pd.read_csv('data/tx_schools.csv', index_col=0)
schools = pd.concat([schools_geo, schools], axis=1)

gdf_aisd = schools[schools['School District'] == 'Austin ISD'].copy().reset_index(drop=True)
gdf_aisd = gdf_aisd[~(gdf_aisd.geometry == None)].copy()


Generate TIF files for each school

In [15]:
# generate_tif_files(gdf_aisd, schools_folder, resolution=LOW_RES_IMAGE_RESOLUTION)\
# tifs = [f for f in os.listdir(schools_folder) if f.endswith('.tif')]
# for tif in tqdm(tifs):
#     tif_to_png(schools_folder, tif)

Generate TIF files for each building located in each school (this is achieved using geosam in another studio)

In [17]:
geojson_folder = os.path.join(schools_folder, 'building_geos')
geojsons = [f for f in os.listdir(geojson_folder) if f.endswith('.geojson')]

In [22]:
schools_folder

'/home/Ryan.Skene/rooftop/data/images/schools'

In [26]:
pbar = tqdm(geojsons, total=len(geojsons))
for geojson in pbar:
    school_name = geojson.split('_')[0]
    pbar.set_description(f'Processing {school_name}')

    gdf = gpd.GeoDataFrame.from_file(os.path.join(geojson_folder, geojson))
    for i, bounds in tqdm(gdf.bounds.iterrows(), total=len(gdf), leave=False):
        tif_filename = f'{school_name} Building {i}.tif'
        bbox = bounds.values.tolist()

        leafmap.map_tiles_to_geotiff(
            output=os.path.join(schools_folder, tif_filename), bbox=bbox, resolution=HIGH_RES_IMAGE_RESOLUTION, source="Satellite", overwrite=True
        )
        tif_to_png(schools_folder, tif_filename)

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Downloaded image 1/357
Downloaded image 2/357
Downloaded image 3/357
Downloaded image 4/357
Downloaded image 5/357
Downloaded image 6/357
Downloaded image 7/357
Downloaded image 8/357
Downloaded image 9/357
Downloaded image 10/357
Downloaded image 11/357
Downloaded image 12/357
Downloaded image 13/357
Downloaded image 14/357
Downloaded image 15/357
Downloaded image 16/357
Downloaded image 17/357
Downloaded image 18/357
Downloaded image 19/357
Downloaded image 20/357
Downloaded image 21/357
Downloaded image 22/357
Downloaded image 23/357
Downloaded image 24/357
Downloaded image 25/357
Downloaded image 26/357
Downloaded image 27/357
Downloaded image 28/357
Downloaded image 29/357
Downloaded image 30/357
Downloaded image 31/357
Downloaded image 32/357
Downloaded image 33/357
Downloaded image 34/357
Downloaded image 35/357
Downloaded image 36/357
Downloaded image 37/357
Downloaded image 38/357
Downloaded image 39/357
Downloaded image 40/357
Downloaded image 41/357
Downloaded image 42/357
D

  0%|          | 0/12 [00:00<?, ?it/s]

Downloaded image 1/143
Downloaded image 2/143
Downloaded image 3/143
Downloaded image 4/143
Downloaded image 5/143
Downloaded image 6/143
Downloaded image 7/143
Downloaded image 8/143
Downloaded image 9/143
Downloaded image 10/143
Downloaded image 11/143
Downloaded image 12/143
Downloaded image 13/143
Downloaded image 14/143
Downloaded image 15/143
Downloaded image 16/143
Downloaded image 17/143
Downloaded image 18/143
Downloaded image 19/143
Downloaded image 20/143
Downloaded image 21/143
Downloaded image 22/143
Downloaded image 23/143
Downloaded image 24/143
Downloaded image 25/143
Downloaded image 26/143
Downloaded image 27/143
Downloaded image 28/143
Downloaded image 29/143
Downloaded image 30/143
Downloaded image 31/143
Downloaded image 32/143
Downloaded image 33/143
Downloaded image 34/143
Downloaded image 35/143
Downloaded image 36/143
Downloaded image 37/143
Downloaded image 38/143
Downloaded image 39/143
Downloaded image 40/143
Downloaded image 41/143
Downloaded image 42/143
D

Convert each building TIF to png for use in Roboflow inference